In [9]:
import osmnx as ox
from geopy.geocoders import Nominatim
import folium
import networkx as nx
import json
import webbrowser
import tkinter as tk
from Algorithm import AStar # Import class AStar từ file Algorithm.py
from tkinter import ttk
from tkinter import messagebox # Để hiển thị thông báo
from osmnx.convert import to_undirected # Import hàm to_undirected từ module convert

# Tải graph từ OpenStreetMap (theo tên khu vực)
G = ox.graph_from_place("Tây Hồ, Hà Nội, Việt Nam", network_type="drive")

# Thay thế dòng này:
# G = get_undirected(G)
# Bằng dòng gọi hàm to_undirected đã import:
G = to_undirected(G) # Chuyển đổi đồ thị G sang dạng vô hướng

G_proj = ox.project_graph(G)

# 2. Geocoder
geolocator = Nominatim(user_agent="my_mapper", domain="nominatim.openstreetmap.org")

# Tiếp tục với phần còn lại của code của bạn

In [10]:
nodes, edges = ox.graph_to_gdfs(G)

print(edges.head())

                                             osmid       highway   lanes  \
u         v           key                                                  
103027089 4798381069  0                 1336409910  primary_link       1   
          4387463104  0                  702522171       primary       2   
          1756929010  0                  506316676       primary       2   
103027166 2294341609  0                  702554085   residential     NaN   
          12578123264 0    [1374912738, 702501614]       primary  [3, 2]   

                           oneway reversed      length        from  \
u         v           key                                            
103027089 4798381069  0     False    False   11.854081  4798381069   
          4387463104  0      True    False  248.104179   103027089   
          1756929010  0      True    False   38.618307  1756929010   
103027166 2294341609  0     False     True   12.297765  2294341609   
          12578123264 0      True    False   88

In [11]:
def get_nearest_node(G, address):
    full_address = f"{address}, Tây Hồ, Hà Nội, Việt Nam"
    location = geolocator.geocode(full_address)
    if location is None:
        raise Exception(f"Không tìm thấy địa chỉ: {address}")
    
    lat = location.latitude
    lon = location.longitude
    
    # Get the nearest node
    nearest_node = ox.distance.nearest_nodes(G, X=lon, Y=lat)
    
    # Check if this node is part of a two-way road
    # Get all edges connected to this node
    connected_edges = list(G.edges(nearest_node, keys=True))
    
    # If there are multiple edges with different keys between the same nodes,
    # it means it's a two-way road
    for u, v, k in connected_edges:
        if G.has_edge(v, u):  # Check if there's an edge in the opposite direction
            # Return both nodes to consider both directions
            return nearest_node, v
    
    return nearest_node


In [12]:
def draw_and_find_shortest_path(G, start_node, end_node):
    # Hà Nội: Tây Hồ
    center_lat = 21.08
    center_lon = 105.82
    m = folium.Map(location=[center_lat, center_lon], zoom_start=15, tiles='OpenStreetMap')

    # --- Vẽ tất cả các nodes ---
    for node_id, node_data in G.nodes(data=True):
        lat = node_data.get('y')
        lon = node_data.get('x')
        if lat and lon:
            folium.CircleMarker(
                location=[lat, lon],
                radius=2,
                color='red',
                fill=True,
                fill_opacity=0.7
            ).add_to(m)

    # --- Vẽ tất cả các edges ---
    for u, v in G.edges():
        u_data = G.nodes[u]
        v_data = G.nodes[v]
        points = [
            [u_data['y'], u_data['x']],
            [v_data['y'], v_data['x']]
        ]
        folium.PolyLine(points, color='gray', weight=1, opacity=0.6).add_to(m)

    # --- Vẽ đường đi ngắn nhất --- 
    source_id = start_node
    target_id = end_node
    coordinates = {node: (data['y'], data['x']) for node, data in G.nodes(data=True)}  # Lưu tọa độ nodes
    try:
        astar = AStar(G, coordinates)  # Khởi tạo đối tượng AStar
        path, total_cost  = astar.find_path(start_node, end_node)  # Tìm đường đi ngắn nhất
        path_coords = [[G.nodes[n]['y'], G.nodes[n]['x']] for n in path]

        folium.PolyLine(
            path_coords,
            color='blue',
            weight=4,
            opacity=0.9,
            tooltip='Đường đi ngắn nhất'
        ).add_to(m)

        # Đánh dấu điểm đầu/cuối
        folium.Marker(location=path_coords[0], popup='Điểm bắt đầu', icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(location=path_coords[-1], popup='Điểm kết thúc', icon=folium.Icon(color='blue')).add_to(m)

    except nx.NetworkXNoPath:
        print("Không tìm thấy đường đi giữa 2 node.")

    # --- Hiển thị bản đồ ---
    # Lưu bản đồ vào file tạm và mở bằng trình duyệt
    m.save("ban_do_ha_noi.html") 
    print(f"Chi phí đường đi: {total_cost}")
    print("da thuc hien ham nay")

In [13]:
import random

def simulate_traffic_and_draw(G, start_node, end_node):
    center_lat = 21.08
    center_lon = 105.82
    m = folium.Map(location=[center_lat, center_lon], zoom_start=15, tiles='OpenStreetMap')

    coordinates = {node: (data['y'], data['x']) for node, data in G.nodes(data=True)}
    astar = AStar(G, coordinates)

    # 1. Đường đi ban đầu
    path, cost = astar.find_path(start_node, end_node)
    if not path:
        print("Không tìm thấy đường đi ban đầu.")
        return

    # Vẽ đường ban đầu màu cam
    path_coords = [[G.nodes[n]['y'], G.nodes[n]['x']] for n in path]
    folium.PolyLine(path_coords, color='orange', weight=4, tooltip='Đường ban đầu').add_to(m)

    # 2. Random một node để giả lập ngập lụt
    # Lấy danh sách các nút trong đồ thị, loại trừ điểm bắt đầu và kết thúc
    eligible_nodes = [n for n in G.nodes() if n not in [start_node, end_node]]

    if not eligible_nodes:
        print("Không có nút nào đủ điều kiện để giả lập ngập lụt (trừ điểm đầu/cuối).")
        # Vẫn hiển thị bản đồ với đường đi ban đầu và điểm đầu cuối
        folium.Marker(location=path_coords[0], popup='Điểm bắt đầu', icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(location=path_coords[-1], popup='Điểm kết thúc', icon=folium.Icon(color='blue')).add_to(m)
        map_filename = "ban_do_ha_noi.html"
        m.save(map_filename)
        webbrowser.open(map_filename)
        return

    # Chọn ngẫu nhiên một nút từ danh sách đủ điều kiện
    random_node = random.choice(eligible_nodes)
    print(f"Giả lập ngập lụt tại nút ngẫu nhiên: {random_node}")

    # Lấy tất cả các cạnh incident (nối trực tiếp) với nút random_node
    # G.edges(node, data=True, keys=True) trả về iterator của (u, v, key, data)
    affected_edges_data = list(G.edges(random_node, data=True, keys=True))

    if not affected_edges_data:
        print(f"Nút {random_node} không có cạnh nào được kết nối.")
        # Vẫn hiển thị bản đồ với đường đi ban đầu và điểm đầu cuối
        folium.Marker(location=path_coords[0], popup='Điểm bắt đầu', icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(location=path_coords[-1], popup='Điểm kết thúc', icon=folium.Icon(color='blue')).add_to(m)
        map_filename = "ban_do_ha_noi.html"
        m.save(map_filename)
        webbrowser.open(map_filename)
        return

    # Vẽ các cạnh bị ảnh hưởng màu đỏ (những chỗ bị lụt) và tăng trọng số
    print(f"Vẽ các cạnh lân cận của nút {random_node} (giả lập ngập lụt) và tăng trọng số:")
    edges_to_modify = []
    for u_edge, v_edge, k_edge, data_edge in affected_edges_data:
        # Lưu thông tin cạnh để sửa đổi sau (tránh sửa đổi khi đang lặp)
        edges_to_modify.append((u_edge, v_edge, k_edge))

        # Vẽ cạnh bị ảnh hưởng màu đỏ
        # Kiểm tra xem nút có tồn tại trong G.nodes() trước khi truy cập tọa độ
        if u_edge in G.nodes() and v_edge in G.nodes():
            edge_coords = [[G.nodes[u_edge]['y'], G.nodes[u_edge]['x']], [G.nodes[v_edge]['y'], G.nodes[v_edge]['x']]]
            folium.PolyLine(edge_coords, color='red', weight=6, tooltip=f'Đoạn bị ngập gần nút: {random_node}').add_to(m)
            print(f"- Cạnh ({u_edge}, {v_edge}, {k_edge})")
        else:
             print(f"  Cảnh báo: Nút {u_edge} hoặc {v_edge} không tồn tại trong đồ thị khi vẽ cạnh bị ngập.")


    # Tăng trọng số các cạnh đã xác định trong đồ thị G (bản sao)
    increase_factor = 100 # Hệ số tăng trọng số
    for u, v, k in edges_to_modify:
         # Kiểm tra thuộc tính 'length' tồn tại trước khi tăng
         if 'length' in G[u][v][k]:
             G[u][v][k]['length'] *= increase_factor
             # print(f"  Tăng trọng số cạnh ({u}, {v}, {k}) lên {increase_factor} lần.") # Có thể in ra để kiểm tra
         else:
             # Nếu không có thuộc tính 'length', thêm nó vào với giá trị ban đầu là 1 và tăng lên
             G[u][v][k]['length'] = 1 * increase_factor
             # print(f"  Thêm thuộc tính 'length' cho cạnh ({u}, {v}, {k}) và tăng lên {increase_factor} lần.")


    # Đánh dấu nút bị ngập lụt
    if random_node in G.nodes():
         folium.CircleMarker(
             location=[G.nodes[random_node]['y'], G.nodes[random_node]['x']],
             radius=8,
             color='darkred',
             fill=True,
             fill_color='red',
             fill_opacity=0.7,
             tooltip=f'Nút bị ngập: {random_node}'
         ).add_to(m)
    else:
        print(f"Cảnh báo: Nút ngẫu nhiên {random_node} không tồn tại trong đồ thị khi vẽ điểm ngập.")

    # 3. Tìm đường mới
    new_path, new_cost = astar.find_path(start_node, end_node)
    if new_path:
        new_coords = [[G.nodes[n]['y'], G.nodes[n]['x']] for n in new_path]
        folium.PolyLine(new_coords, color='green', weight=4, tooltip='Đường mới').add_to(m)
    else:
        print("Không tìm được đường thay thế.")

    # Đánh dấu điểm đầu/cuối
    folium.Marker(location=path_coords[0], popup='Điểm bắt đầu', icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=path_coords[-1], popup='Điểm kết thúc', icon=folium.Icon(color='blue')).add_to(m)

    m.save("ban_do_ha_noi.html")
    webbrowser.open("ban_do_ha_noi.html")

In [14]:
def simulate_flooding_and_draw(G_input, start_node, end_node):
    G = G_input.copy()
    center_lat, center_lon = 21.08, 105.82

    coordinates = {node: (data['y'], data['x']) for node, data in G.nodes(data=True)}
    astar = AStar(G, coordinates)

    print(f"Tìm đường đi ban đầu từ {start_node} đến {end_node}...")
    path, cost = astar.find_path(start_node, end_node)

    m = folium.Map(location=[center_lat, center_lon], zoom_start=14, tiles='OpenStreetMap')

    if not path:
        print(f"Không tìm thấy đường đi ban đầu.")
        if start_node in G.nodes():
            folium.Marker(location=[G.nodes[start_node]['y'], G.nodes[start_node]['x']], icon=folium.Icon(color='green'), popup='Điểm bắt đầu').add_to(m)
        if end_node in G.nodes():
            folium.Marker(location=[G.nodes[end_node]['y'], G.nodes[end_node]['x']], icon=folium.Icon(color='blue'), popup='Điểm kết thúc').add_to(m)
        m.save("ban_do_ha_noi.html")
        webbrowser.open("ban_do_ha_noi.html")
        return

    print(f"Tìm thấy đường đi ban đầu với chi phí: {cost:.2f}")
    path_coords = [[G.nodes[n]['y'], G.nodes[n]['x']] for n in path]
    folium.PolyLine(path_coords, color='orange', weight=4, tooltip=f'Đường ban đầu (Chi phí: {cost:.2f})').add_to(m)

    # Chọn ngẫu nhiên một nút bị ngập
    flooded_node = random.choice(list(G.nodes()))
    print(f"Giả lập ngập tại nút: {flooded_node}")

    affected_edges_data = list(G.edges(flooded_node, data=True, keys=True))
    edges_to_modify = []

    def draw_flooded_edge(u, v, k):
        if u in G.nodes() and v in G.nodes():
            coords = [[G.nodes[u]['y'], G.nodes[u]['x']], [G.nodes[v]['y'], G.nodes[v]['x']]]
            folium.PolyLine(coords, color='red', weight=6, tooltip=f'Đoạn bị ngập ({u}-{v})').add_to(m)

    print(f"-> Cạnh trực tiếp với {flooded_node}:")
    for u, v, k, data in affected_edges_data:
        edges_to_modify.append((u, v, k))
        draw_flooded_edge(u, v, k)
        print(f"- ({u}, {v}, {k})")

    neighbors = list(G.neighbors(flooded_node))
    print(f"-> Các nút láng giềng của {flooded_node}: {neighbors}")

    for neighbor in neighbors:
        neighbor_edges = list(G.edges(neighbor, data=True, keys=True))
        print(f"  -> Cạnh của nút láng giềng {neighbor}:")
        for u, v, k, data in neighbor_edges:
            if (u, v, k) not in edges_to_modify and (v, u, k) not in edges_to_modify:
                edges_to_modify.append((u, v, k))
                draw_flooded_edge(u, v, k)
                print(f"  - ({u}, {v}, {k})")

    # Tăng trọng số các cạnh đã xác định
    increase_factor = 100
    for u, v, k in edges_to_modify:
        if 'length' in G[u][v][k]:
            G[u][v][k]['length'] *= increase_factor

    # Đánh dấu nút ngập
    folium.CircleMarker(
        location=[G.nodes[flooded_node]['y'], G.nodes[flooded_node]['x']],
        radius=8,
        color='darkred',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        tooltip=f'Nút bị ngập: {flooded_node}'
    ).add_to(m)

    # Tìm đường mới
    print("Tìm đường mới sau khi ngập...")
    new_path, new_cost = astar.find_path(start_node, end_node)

    if new_path:
        print(f"Đường mới với chi phí: {new_cost:.2f}")
        new_coords = [[G.nodes[n]['y'], G.nodes[n]['x']] for n in new_path]
        folium.PolyLine(new_coords, color='green', weight=4, tooltip=f'Đường mới (Chi phí: {new_cost:.2f})').add_to(m)
    else:
        print("Không tìm thấy đường thay thế sau ngập lụt.")

    # Vẽ marker đầu/cuối
    folium.Marker(location=path_coords[0], popup='Điểm bắt đầu', icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=path_coords[-1], popup='Điểm kết thúc', icon=folium.Icon(color='blue')).add_to(m)

    map_filename = "ban_do_ha_noi.html"
    m.save(map_filename)
    print(f"Đã lưu bản đồ vào {map_filename}")
    webbrowser.open(map_filename)

# NẾU CẦN PHẢI LỰA CHỌN NÚT BỊ LỤT THÌ SỬ DỤNG HÀM BÊN DƯỚI (gpt để dc định dạng chuẩn)

# def simulate\_flooding\_and\_draw(G\_input, start\_node, end\_node, flooded\_node\_id):

# ```
# G = G_input.copy()
# center_lat, center_lon = 21.08, 105.82

# coordinates = {node: (data['y'], data['x']) for node, data in G.nodes(data=True)}
# astar = AStar(G, coordinates)

# print(f"Tìm đường đi ban đầu từ {start_node} đến {end_node}...")
# path, cost = astar.find_path(start_node, end_node)

# m = folium.Map(location=[center_lat, center_lon], zoom_start=14, tiles='OpenStreetMap')

# if not path:
#     print(f"Không tìm thấy đường đi ban đầu.")
#     if start_node in G.nodes():
#         folium.Marker(location=[G.nodes[start_node]['y'], G.nodes[start_node]['x']], icon=folium.Icon(color='green'), popup='Điểm bắt đầu').add_to(m)
#     if end_node in G.nodes():
#         folium.Marker(location=[G.nodes[end_node]['y'], G.nodes[end_node]['x']], icon=folium.Icon(color='blue'), popup='Điểm kết thúc').add_to(m)
#     m.save("ban_do_ha_noi_ngap_lut.html")
#     webbrowser.open("ban_do_ha_noi_ngap_lut.html")
#     return

# print(f"Tìm thấy đường đi ban đầu với chi phí: {cost:.2f}")
# path_coords = [[G.nodes[n]['y'], G.nodes[n]['x']] for n in path]
# folium.PolyLine(path_coords, color='orange', weight=4, tooltip=f'Đường ban đầu (Chi phí: {cost:.2f})').add_to(m)

# flooded_node = flooded_node_id
# if flooded_node not in G.nodes():
#     print(f"Nút ngập {flooded_node} không tồn tại.")
#     folium.Marker(location=path_coords[0], popup='Điểm bắt đầu', icon=folium.Icon(color='green')).add_to(m)
#     folium.Marker(location=path_coords[-1], popup='Điểm kết thúc', icon=folium.Icon(color='blue')).add_to(m)
#     m.save("ban_do_ha_noi_ngap_lut.html")
#     webbrowser.open("ban_do_ha_noi_ngap_lut.html")
#     return

# print(f"Giả lập ngập tại nút: {flooded_node}")
# affected_edges_data = list(G.edges(flooded_node, data=True, keys=True))
# edges_to_modify = []

# def draw_flooded_edge(u, v, k):
#     if u in G.nodes() and v in G.nodes():
#         coords = [[G.nodes[u]['y'], G.nodes[u]['x']], [G.nodes[v]['y'], G.nodes[v]['x']]]
#         folium.PolyLine(coords, color='red', weight=6, tooltip=f'Đoạn bị ngập ({u}-{v})').add_to(m)

# print(f"-> Cạnh trực tiếp với {flooded_node}:")
# for u, v, k, data in affected_edges_data:
#     edges_to_modify.append((u, v, k))
#     draw_flooded_edge(u, v, k)
#     print(f"- ({u}, {v}, {k})")

# neighbors = list(G.neighbors(flooded_node))
# print(f"-> Các nút láng giềng của {flooded_node}: {neighbors}")

# for neighbor in neighbors:
#     neighbor_edges = list(G.edges(neighbor, data=True, keys=True))
#     print(f"  -> Cạnh của nút láng giềng {neighbor}:")
#     for u, v, k, data in neighbor_edges:
#         if (u, v, k) not in edges_to_modify and (v, u, k) not in edges_to_modify:
#             edges_to_modify.append((u, v, k))
#             draw_flooded_edge(u, v, k)
#             print(f"  - ({u}, {v}, {k})")

# # Tăng trọng số các cạnh đã xác định
# increase_factor = 100
# for u, v, k in edges_to_modify:
#     if 'length' in G[u][v][k]:
#         G[u][v][k]['length'] *= increase_factor

# # Đánh dấu nút ngập
# folium.CircleMarker(
#     location=[G.nodes[flooded_node]['y'], G.nodes[flooded_node]['x']],
#     radius=8,
#     color='darkred',
#     fill=True,
#     fill_color='red',
#     fill_opacity=0.7,
#     tooltip=f'Nút bị ngập: {flooded_node}'
# ).add_to(m)

# # Tìm đường mới
# print("Tìm đường mới sau khi ngập...")
# new_path, new_cost = astar.find_path(start_node, end_node)

# if new_path:
#     print(f"Đường mới với chi phí: {new_cost:.2f}")
#     new_coords = [[G.nodes[n]['y'], G.nodes[n]['x']] for n in new_path]
#     folium.PolyLine(new_coords, color='green', weight=4, tooltip=f'Đường mới (Chi phí: {new_cost:.2f})').add_to(m)
# else:
#     print("Không tìm thấy đường thay thế sau ngập lụt.")

# # Vẽ marker đầu/cuối
# folium.Marker(location=path_coords[0], popup='Điểm bắt đầu', icon=folium.Icon(color='green')).add_to(m)
# folium.Marker(location=path_coords[-1], popup='Điểm kết thúc', icon=folium.Icon(color='blue')).add_to(m)

# map_filename = "ban_do_ha_noi_ngap_lut.html"
# m.save(map_filename)
# print(f"Đã lưu bản đồ vào {map_filename}")
# webbrowser.open(map_filename) 



In [15]:
class InputAppWithModeAndButton:
    def __init__(self, root):
        self.root = root
        # --- Đặt tiêu đề cửa sổ bằng tiếng Việt ---
        self.root.title("Tìm Đường Đơn Giản")
        self.root.geometry("430x220") # Điều chỉnh kích thước nếu cần
        self.simulate_traffic_var = tk.BooleanVar(value=False)
        
        # --- Khung chứa địa điểm Bắt đầu và Kết thúc ---
        input_frame = ttk.Frame(root, padding="15 15 15 5")
        input_frame.pack(fill=tk.X, padx=10, pady=(10, 5))
        input_frame.columnconfigure(0, weight=0)
        input_frame.columnconfigure(1, weight=1)

        # --- Nhãn và ô nhập Địa điểm bắt đầu bằng tiếng Việt ---
        start_label = ttk.Label(input_frame, text="Điểm bắt đầu:")
        start_label.grid(row=0, column=0, padx=(0, 5), pady=5, sticky=tk.W)
        self.start_entry = ttk.Entry(input_frame, width=40)
        self.start_entry.grid(row=0, column=1, padx=5, pady=5, sticky=tk.EW)
        # Đã xóa dòng: self.start_entry.insert(0, "Hà Nội")

        # --- Nhãn và ô nhập Địa điểm kết thúc bằng tiếng Việt ---
        end_label = ttk.Label(input_frame, text="Điểm kết thúc:")
        end_label.grid(row=1, column=0, padx=(0, 5), pady=5, sticky=tk.W)
        self.end_entry = ttk.Entry(input_frame, width=40)
        self.end_entry.grid(row=1, column=1, padx=5, pady=5, sticky=tk.EW)
        # Đã xóa dòng: self.end_entry.insert(0, "Ninh Bình")

        # --- Khung chứa lựa chọn Phương tiện di chuyển ---
        mode_frame = ttk.Frame(root, padding="15 5 15 5")
        mode_frame.pack(fill=tk.X, padx=10, pady=5)

        # --- Nhãn Phương tiện bằng tiếng Việt ---
        ttk.Label(mode_frame, text="Phương tiện:").pack(side=tk.LEFT, padx=(0, 10), pady=5)

        # Biến để lưu lựa chọn phương tiện
        self.mode_var = tk.StringVar(value="driving") # Giá trị mặc định

        # --- Các nút RadioButton với text tiếng Việt ---
        modes = [("Lái xe", "driving"), ("Đi bộ", "walking"), ("Đạp xe", "cycling")]
        for text, mode_value in modes:
            rb = ttk.Radiobutton(mode_frame,
                                 text=text,
                                 variable=self.mode_var,
                                 value=mode_value)
            rb.pack(side=tk.LEFT, padx=5)

        # --- Khung chứa nút bấm ---
        button_frame = ttk.Frame(root, padding="15 5 15 15")
        button_frame.pack(fill=tk.X, padx=10, pady=(5, 10))
        button_frame.columnconfigure(0, weight=1) # Căn giữa nút

        # --- Nút bấm với text tiếng Việt ---
        self.find_button = ttk.Button(button_frame, text="Tìm đường", command=self.process_input)
        self.find_button.grid(row=0, column=0, pady=5)
           # --- Checkbox để giả lập tắc đường ---
        self.simulate_traffic_var = tk.BooleanVar(value=False)
        simulate_checkbox = ttk.Checkbutton(button_frame,
                                            text="Giả lập tắc đường",
                                            variable=self.simulate_traffic_var)
        simulate_checkbox.grid(row=1, column=0, pady=5)

        # --- Checkbox để giả lập lũ lụt ---
        self.simulate_flood_var = tk.BooleanVar(value=False)
        simulate_flood_checkbox = ttk.Checkbutton(button_frame,
                                          text="Giả lập lũ lụt",
                                          variable=self.simulate_flood_var)
        simulate_flood_checkbox.grid(row=2, column=0, pady=5)

        self.start_entry.focus() # Đặt con trỏ vào ô nhập đầu tiên
        
    def process_input(self):
        start_loc = self.start_entry.get().strip()
        end_loc = self.end_entry.get().strip()
        selected_mode = self.mode_var.get()

        if not start_loc or not end_loc:
            messagebox.showwarning("Thiếu thông tin", "Vui lòng nhập cả địa điểm bắt đầu và kết thúc.")
            return

        print("--- Dữ liệu đã nhận ---")
        print(f"Điểm bắt đầu: {start_loc}")
        print(f"Điểm kết thúc: {end_loc}")
        print(f"Phương tiện:   {selected_mode}")
        print(f"Tắc đường:     {self.simulate_traffic_var.get()}")
        print(f"Lũ lụt:        {self.simulate_flood_var.get()}")
        print("----------------------")

        start_node = get_nearest_node(G, start_loc)
        end_node = get_nearest_node(G, end_loc)
        if isinstance(start_node, tuple):
            start_node = start_node[0]
        if isinstance(end_node, tuple):
            end_node = end_node[0]

        if self.simulate_flood_var.get():
            simulate_flooding_and_draw(G, start_node, end_node)
        elif self.simulate_traffic_var.get():
            simulate_traffic_and_draw(G, start_node, end_node)
        else:
            draw_and_find_shortest_path(G, start_node, end_node)

   

In [16]:
if __name__ == "__main__":
    main_window = tk.Tk()
    app = InputAppWithModeAndButton(main_window)
    main_window.mainloop()


--- Dữ liệu đã nhận ---
Điểm bắt đầu: võ chí công
Điểm kết thúc: thụy khuê
Phương tiện:   driving
Tắc đường:     True
Lũ lụt:        True
----------------------
Tìm đường đi ban đầu từ 8298863971 đến 7870785197...
Tìm thấy đường đi ban đầu với chi phí: 5740.37
Giả lập ngập tại nút: 309818642
-> Cạnh trực tiếp với 309818642:
- (309818642, 8329563308, 0)
- (309818642, 320101381, 0)
- (309818642, 5687140498, 0)
-> Các nút láng giềng của 309818642: [8329563308, 320101381, 5687140498]
  -> Cạnh của nút láng giềng 8329563308:
  - (8329563308, 8329563295, 0)
  -> Cạnh của nút láng giềng 320101381:
  - (320101381, 311718209, 0)
  - (320101381, 8329563295, 0)
  -> Cạnh của nút láng giềng 5687140498:
  - (5687140498, 310725747, 0)
Tìm đường mới sau khi ngập...
Đường mới với chi phí: 5740.37
Đã lưu bản đồ vào ban_do_ha_noi.html
--- Dữ liệu đã nhận ---
Điểm bắt đầu: võ chí công
Điểm kết thúc: thụy khuê
Phương tiện:   driving
Tắc đường:     True
Lũ lụt:        False
----------------------
Giả lập n